In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

Подробное описание оконных функций в SQLite можно посмотреть [на офф. сайте](https://sqlite.org/windowfunctions.html)

Оконная функция - это функция, выполняемая в рамках определенного окна, специфицирующимся в блоке OVER( ).
Оконные функции подразделяются на агрегатные оконные функции и встроенные оконные функции. 



In [2]:
%%sql 
select count(*) 
from tracks


 * sqlite:///chinook.db
Done.


count(*)
3503


Проверяем корректность версии


Для возможности работы с оконными функциями версия должна быть больше 3.25.0

In [6]:
%%sql
select SQLITE_VERSION() 

 * sqlite:///chinook.db
Done.


SQLITE_VERSION()
3.31.1


## Агрегатные оконные функции

```
SELECT <название агрегатной функции>(поле) [FILTER (WHERE <выражение>)] OVER (<определение окна>)

```

Вывести самые дешевые треки по жанру

In [ ]:
%%sql 
select gname, price, tname
from
(
select g.name as gname, row_number()
    over (partition by g.genreid order by t.unitprice asc) rn,
     t.unitprice as price, t.name as tname
  from tracks t
  join genres g
    on t.genreid = g.genreid 
)
where rn = 1

In [ ]:
%%sql 
select gname, price, tname
from
(
select g.name as gname, rank()
    over (partition by g.genreid order by t.unitprice asc) rnk,
     t.unitprice as price, t.name as tname
  from tracks t
  join genres g
    on t.genreid = g.genreid 
)
where rnk = 1

In [ ]:
%%sql 
select distinct g.name as gname, first_value(t.name)
    over (partition by g.genreid order by t.unitprice asc) trackName,
     first_value(t.unitprice)
    over (partition by g.genreid order by t.unitprice asc) unitPrice
     
  from tracks t
  join genres g
    on t.genreid = g.genreid 


Для каждого альбома определить жанр альбома и среднюю продолжительность треков 

In [ ]:
%%sql 
WITH a as
(
select a.title,
       g.name, 
       count(*) tr_gn_cnt,
       avg(t.milliseconds) ms
  from tracks t
  join genres g 
    on t.genreid = g.genreid
  join albums a 
    on a.albumid = t.albumid
group by a.albumid, a.title, g.genreid, g.name
)

select title, name, ms
from
(
    select title, 
           name, 
           ms,
           row_number() over (partition by title, name 
                              order by tr_gn_cnt desc) rn 
      from a 
)
where rn = 1
    

In [ ]:
%%sql 
WITH a as
(
select a.title,
       g.name, 
       count(*) tr_gn_cnt,
       avg(t.milliseconds) ms
  from tracks t
  join genres g 
    on t.genreid = g.genreid
  join albums a 
    on a.albumid = t.albumid
group by a.albumid, a.title, g.genreid, g.name
)

    select distinct title, 
           name, 
           
           first_value(ms) over (partition by title, name 
                              order by tr_gn_cnt desc) rn 
      from a 

Для каждой даты, в которую был нанят сотрудник вывести количество нанятых сотрудников до этой даты включительно.

In [ ]:
%%sql
Select distinct hiredate, count(*) over (order by date(hiredate) 
                                rows between unbounded preceding
                                and current row) cntHired
from employees 